# NBA Props Model Evaluation - PRA Predictions for Betting

## Objective
Build and evaluate machine learning models for predicting NBA player PRA (Points + Rebounds + Assists) with a focus on practical betting applications.

### Key Considerations for Sports Betting:
- **Temporal validation**: Sports data is time-dependent
- **Calibration**: Need well-calibrated probabilities for over/under betting
- **Confidence intervals**: Critical for risk management
- **Line coverage**: Accuracy at different betting lines (e.g., 20.5, 25.5, 30.5)
- **Edge identification**: Finding mispriced lines with statistical confidence

### Model Requirements:
1. Point predictions for PRA values
2. Prediction intervals (confidence bounds)
3. Probability estimates for over/under specific lines
4. Feature importance for understanding key drivers
5. Performance stability across player types

In [1]:
# Import required libraries
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Modeling libraries
from sklearn.model_selection import TimeSeriesSplit, cross_val_score, KFold
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import Ridge, Lasso, ElasticNet, HuberRegressor, QuantileRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import SVR
import xgboost as xgb
import lightgbm as lgb
from sklearn.neural_network import MLPRegressor

# Statistical libraries
from scipy import stats
from scipy.stats import norm, t
from sklearn.isotonic import IsotonicRegression
from sklearn.calibration import calibration_curve

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Set visualization style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

print("Libraries loaded successfully")
print(f"Python version: {pd.__version__}")
print(f"XGBoost version: {xgb.__version__}")
print(f"LightGBM version: {lgb.__version__}")

Libraries loaded successfully
Python version: 2.3.2
XGBoost version: 3.0.5
LightGBM version: 4.6.0


## 1. Data Loading and Preparation

In [2]:
# Load processed features
data_path = Path('/Users/diyagamah/Documents/nba_props_model/data/processed')
df = pd.read_csv(data_path / 'player_features_2023_24.csv')

print(f"Dataset shape: {df.shape}")
print(f"Players: {len(df)}")
print(f"Features: {len(df.columns) - 1}")
print("\nTarget variable (PRA_estimate) statistics:")
print(df['PRA_estimate'].describe())

# Display feature columns
feature_cols = [col for col in df.columns if col not in ['Player', 'Team', 'PRA_estimate', 
                                                          'Points_estimate', 'Rebounds_estimate', 
                                                          'Assists_estimate']]
print(f"\nInput features ({len(feature_cols)}):")
for i, col in enumerate(feature_cols, 1):
    print(f"{i:2d}. {col}")

Dataset shape: (503, 27)
Players: 503
Features: 26

Target variable (PRA_estimate) statistics:
count    503.000000
mean       8.927022
std        7.892180
min        0.339459
25%        2.777252
50%        6.547002
75%       13.183307
max       42.964009
Name: PRA_estimate, dtype: float64

Input features (21):
 1. USG_percent
 2. PSA
 3. MIN
 4. AST_percent
 5. AST_to_USG_Ratio
 6. fgDR_percent
 7. fgOR_percent
 8. Total_REB_percent
 9. eFG_percent
10. TOV_percent
11. Defensive_Activity
12. Position_Inferred
13. Player_Role
14. Consistency_Score
15. Usage_Stability
16. Performance_Tier
17. Opportunity_Score
18. Efficiency_x_Volume
19. Playmaking_Efficiency
20. Minutes_x_Efficiency
21. Offensive_Load


In [3]:
# Prepare features and target
X = df[feature_cols].copy()
y = df['PRA_estimate'].copy()

# Store player metadata for analysis
player_info = df[['Player', 'Team', 'MIN', 'Performance_Tier']].copy()

# Check for missing values
print("Missing values per feature:")
missing = X.isnull().sum()
if missing.sum() > 0:
    print(missing[missing > 0])
else:
    print("No missing values found")

# Basic statistics
print(f"\nTarget range: {y.min():.2f} - {y.max():.2f}")
print(f"Target mean: {y.mean():.2f}")
print(f"Target std: {y.std():.2f}")
print(f"Target CV: {y.std()/y.mean():.2%}")

Missing values per feature:
No missing values found

Target range: 0.34 - 42.96
Target mean: 8.93
Target std: 7.89
Target CV: 88.41%


## 2. Train-Test Split Strategy

For sports betting, we need to simulate real-world conditions:
- **Temporal validation**: Later games predict future games
- **Player-based split**: Some players for training, others for testing
- **Stratified by performance tier**: Ensure all player types in both sets

In [4]:
# Add the categorical handler import
import sys
sys.path.append('/Users/diyagamah/Documents/nba_props_model/src')
from preprocessing.categorical_handler import CategoricalFeatureHandler, get_model_specific_features

# Strategy 1: Random split stratified by performance tier
from sklearn.model_selection import train_test_split

# Create stratification variable based on PRA quantiles
df['PRA_tier'] = pd.qcut(y, q=5, labels=['Very Low', 'Low', 'Medium', 'High', 'Elite'])

# Identify categorical and numeric columns
categorical_features = ['Position_Inferred', 'Player_Role']
numeric_features = [col for col in feature_cols if col not in categorical_features]

print(f"Categorical features: {categorical_features}")
print(f"Numeric features ({len(numeric_features)}): {numeric_features[:5]}...")

# Split data
X_train, X_test, y_train, y_test, idx_train, idx_test = train_test_split(
    X, y, df.index, test_size=0.2, random_state=42, stratify=df['PRA_tier']
)

# Get player info for test set
test_players = player_info.iloc[idx_test].copy()
test_players['actual_PRA'] = y_test.values

print("\nTrain-Test Split Summary:")
print(f"Training set: {len(X_train)} players")
print(f"Test set: {len(X_test)} players")
print(f"\nPRA distribution in sets:")
print(f"Train - Mean: {y_train.mean():.2f}, Std: {y_train.std():.2f}")
print(f"Test  - Mean: {y_test.mean():.2f}, Std: {y_test.std():.2f}")

# Verify stratification
print("\nTier distribution:")
train_tiers = df.iloc[idx_train]['PRA_tier'].value_counts(normalize=True).sort_index()
test_tiers = df.iloc[idx_test]['PRA_tier'].value_counts(normalize=True).sort_index()
tier_comparison = pd.DataFrame({
    'Train': train_tiers,
    'Test': test_tiers,
    'Difference': test_tiers - train_tiers
})
print(tier_comparison)

Categorical features: ['Position_Inferred', 'Player_Role']
Numeric features (19): ['USG_percent', 'PSA', 'MIN', 'AST_percent', 'AST_to_USG_Ratio']...

Train-Test Split Summary:
Training set: 402 players
Test set: 101 players

PRA distribution in sets:
Train - Mean: 8.85, Std: 7.75
Test  - Mean: 9.24, Std: 8.48

Tier distribution:
             Train      Test  Difference
PRA_tier                                
Very Low  0.201493  0.198020   -0.003473
Low       0.199005  0.198020   -0.000985
Medium    0.201493  0.198020   -0.003473
High      0.199005  0.198020   -0.000985
Elite     0.199005  0.207921    0.008916


In [5]:
# Feature preprocessing with proper categorical handling
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.pipeline import Pipeline

# Create preprocessing pipelines for different model types
print("Creating preprocessing pipelines...")

# For linear models: one-hot encode categoricals and scale numerics
categorical_transformer_linear = OneHotEncoder(sparse_output=False, 
                                              handle_unknown='ignore',
                                              drop='first')
numeric_transformer_linear = RobustScaler()

preprocessor_linear = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer_linear, numeric_features),
        ('cat', categorical_transformer_linear, categorical_features)
    ],
    remainder='passthrough'
)

# For tree models: ordinal encode categoricals, no scaling needed
categorical_transformer_tree = OrdinalEncoder(handle_unknown='use_encoded_value', 
                                             unknown_value=-1)

preprocessor_tree = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', numeric_features),  # No scaling for tree models
        ('cat', categorical_transformer_tree, categorical_features)
    ],
    remainder='passthrough'
)

# Fit and transform for linear models
X_train_linear = preprocessor_linear.fit_transform(X_train)
X_test_linear = preprocessor_linear.transform(X_test)

# Fit and transform for tree models  
X_train_tree = preprocessor_tree.fit_transform(X_train)
X_test_tree = preprocessor_tree.transform(X_test)

# Also create scaled numeric-only versions for comparison
scaler = RobustScaler()
X_train_numeric_scaled = X_train[numeric_features].copy()
X_test_numeric_scaled = X_test[numeric_features].copy()
X_train_numeric_scaled = scaler.fit_transform(X_train_numeric_scaled)
X_test_numeric_scaled = scaler.transform(X_test_numeric_scaled)

print(f"Preprocessing completed:")
print(f"  Linear models shape: {X_train_linear.shape}")
print(f"  Tree models shape: {X_train_tree.shape}")
print(f"  Original shape: {X_train.shape}")

# Get feature names after transformation
linear_feature_names = (numeric_features + 
                       [f"{cat}_{val}" for cat, vals in 
                        zip(categorical_features, preprocessor_linear.named_transformers_['cat'].categories_)
                        for val in vals[1:]])  # Skip first due to drop='first'

tree_feature_names = numeric_features + categorical_features

print(f"\nFeature counts:")
print(f"  Original: {len(feature_cols)}")
print(f"  After linear preprocessing: {len(linear_feature_names)}")
print(f"  After tree preprocessing: {len(tree_feature_names)}")

Creating preprocessing pipelines...
Preprocessing completed:
  Linear models shape: (402, 23)
  Tree models shape: (402, 21)
  Original shape: (402, 21)

Feature counts:
  Original: 21
  After linear preprocessing: 23
  After tree preprocessing: 21


## 3. Model Selection and Training

We'll evaluate multiple model types:
1. **Linear Models**: Ridge, Lasso, ElasticNet, Huber (robust)
2. **Tree-based**: Random Forest, XGBoost, LightGBM, Extra Trees
3. **Other**: SVR, Neural Network
4. **Quantile Regression**: For prediction intervals

In [6]:
# Define models
models = {
    # Linear models
    'Ridge': Ridge(alpha=1.0, random_state=42),
    'Lasso': Lasso(alpha=0.1, random_state=42, max_iter=2000),
    'ElasticNet': ElasticNet(alpha=0.1, l1_ratio=0.5, random_state=42, max_iter=2000),
    'Huber': HuberRegressor(epsilon=1.35, max_iter=200),
    
    # Tree-based models
    'RandomForest': RandomForestRegressor(
        n_estimators=200, max_depth=15, min_samples_split=5,
        min_samples_leaf=2, random_state=42, n_jobs=-1
    ),
    'XGBoost': xgb.XGBRegressor(
        n_estimators=200, max_depth=6, learning_rate=0.05,
        subsample=0.8, colsample_bytree=0.8, random_state=42,
        objective='reg:squarederror', n_jobs=-1
    ),
    'LightGBM': lgb.LGBMRegressor(
        n_estimators=200, max_depth=6, learning_rate=0.05,
        subsample=0.8, colsample_bytree=0.8, random_state=42,
        objective='regression', n_jobs=-1, verbose=-1
    ),
    'ExtraTrees': ExtraTreesRegressor(
        n_estimators=200, max_depth=15, min_samples_split=5,
        min_samples_leaf=2, random_state=42, n_jobs=-1
    ),
    
    # Other models
    'SVR': SVR(kernel='rbf', C=10, gamma='scale', epsilon=0.1),
    'NeuralNet': MLPRegressor(
        hidden_layer_sizes=(100, 50), activation='relu', solver='adam',
        alpha=0.001, max_iter=500, random_state=42, early_stopping=True,
        validation_fraction=0.1
    )
}

print(f"Total models to evaluate: {len(models)}")
print("Models:", list(models.keys()))

Total models to evaluate: 10
Models: ['Ridge', 'Lasso', 'ElasticNet', 'Huber', 'RandomForest', 'XGBoost', 'LightGBM', 'ExtraTrees', 'SVR', 'NeuralNet']


In [7]:
# Train all models and store results
from sklearn.model_selection import cross_val_predict
import time

results = {}
trained_models = {}

# Define which models need which preprocessing
linear_models_list = ['Ridge', 'Lasso', 'ElasticNet', 'Huber', 'SVR', 'NeuralNet']
tree_models_list = ['RandomForest', 'XGBoost', 'LightGBM', 'ExtraTrees']

# Use appropriate data for each model type
for name, model in models.items():
    print(f"\nTraining {name}...")
    start_time = time.time()
    
    # Select appropriate preprocessed data based on model type
    if name in linear_models_list:
        X_train_use = X_train_linear
        X_test_use = X_test_linear
        print(f"  Using linear preprocessing (one-hot encoding + scaling)")
    elif name in tree_models_list:
        X_train_use = X_train_tree
        X_test_use = X_test_tree
        print(f"  Using tree preprocessing (ordinal encoding, no scaling)")
    else:
        # Default to linear preprocessing for safety
        X_train_use = X_train_linear
        X_test_use = X_test_linear
        print(f"  Using linear preprocessing (default)")
    
    # Fit model
    model.fit(X_train_use, y_train)
    trained_models[name] = model
    
    # Make predictions
    y_pred_train = model.predict(X_train_use)
    y_pred_test = model.predict(X_test_use)
    
    # Calculate metrics
    train_mae = mean_absolute_error(y_train, y_pred_train)
    test_mae = mean_absolute_error(y_test, y_pred_test)
    train_rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))
    test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
    train_r2 = r2_score(y_train, y_pred_train)
    test_r2 = r2_score(y_test, y_pred_test)
    
    # Calculate MAPE (Mean Absolute Percentage Error) - useful for betting
    # Avoid division by zero
    mask = y_test != 0
    if mask.sum() > 0:
        test_mape = np.mean(np.abs((y_test[mask] - y_pred_test[mask]) / y_test[mask])) * 100
    else:
        test_mape = np.nan
    
    # Store results
    results[name] = {
        'train_mae': train_mae,
        'test_mae': test_mae,
        'train_rmse': train_rmse,
        'test_rmse': test_rmse,
        'train_r2': train_r2,
        'test_r2': test_r2,
        'test_mape': test_mape,
        'predictions': y_pred_test,
        'training_time': time.time() - start_time
    }
    
    print(f"  MAE: {test_mae:.3f}, RMSE: {test_rmse:.3f}, R²: {test_r2:.3f}, MAPE: {test_mape:.1f}%")
    print(f"  Training time: {results[name]['training_time']:.2f}s")


Training Ridge...
  Using linear preprocessing (one-hot encoding + scaling)
  MAE: 0.639, RMSE: 0.921, R²: 0.988, MAPE: 18.2%
  Training time: 0.01s

Training Lasso...
  Using linear preprocessing (one-hot encoding + scaling)
  MAE: 0.653, RMSE: 1.077, R²: 0.984, MAPE: 12.0%
  Training time: 0.00s

Training ElasticNet...
  Using linear preprocessing (one-hot encoding + scaling)
  MAE: 0.795, RMSE: 1.300, R²: 0.976, MAPE: 19.7%
  Training time: 0.00s

Training Huber...
  Using linear preprocessing (one-hot encoding + scaling)
  MAE: 0.633, RMSE: 0.958, R²: 0.987, MAPE: 17.6%
  Training time: 0.02s

Training RandomForest...
  Using tree preprocessing (ordinal encoding, no scaling)
  MAE: 0.674, RMSE: 1.149, R²: 0.981, MAPE: 8.0%
  Training time: 0.11s

Training XGBoost...
  Using tree preprocessing (ordinal encoding, no scaling)
  MAE: 0.452, RMSE: 0.698, R²: 0.993, MAPE: 5.6%
  Training time: 0.35s

Training LightGBM...
  Using tree preprocessing (ordinal encoding, no scaling)
  MAE: 0

In [8]:
# Create comparison DataFrame
comparison_df = pd.DataFrame(results).T
comparison_df = comparison_df.sort_values('test_mae')

# Display results
print("Model Performance Comparison (sorted by Test MAE):")
print("="*80)
display_cols = ['test_mae', 'test_rmse', 'test_r2', 'test_mape', 'training_time']
print(comparison_df[display_cols].round(3))

# Identify best model
best_model_name = comparison_df.index[0]
print(f"\nBest model: {best_model_name}")
print(f"Test MAE: {comparison_df.loc[best_model_name, 'test_mae']:.3f}")
print(f"Test MAPE: {comparison_df.loc[best_model_name, 'test_mape']:.1f}%")

Model Performance Comparison (sorted by Test MAE):
              test_mae test_rmse   test_r2  test_mape training_time
NeuralNet     0.356451  0.507336  0.996388  10.227402      0.097557
XGBoost       0.452083  0.697788  0.993167   5.621164      0.350354
ExtraTrees    0.536329  0.875046  0.989254   6.785137      0.080443
LightGBM      0.619529  1.325138  0.975356   6.836275      0.308062
Huber         0.633229  0.957894  0.987123  17.619755      0.021648
Ridge         0.639189  0.920885  0.988098  18.245722      0.007924
Lasso          0.65334  1.077078  0.983719   12.02803        0.0039
SVR           0.666495  1.906103   0.94901   18.17554      0.011784
RandomForest  0.673997  1.148638  0.981484   8.049935      0.114255
ElasticNet    0.794785  1.300329   0.97627   19.74051      0.001805

Best model: NeuralNet
Test MAE: 0.356
Test MAPE: 10.2%


## 4. Cross-Validation with Multiple Strategies

In [9]:
# Perform robust cross-validation for top models
from sklearn.pipeline import Pipeline

top_models = comparison_df.head(3).index.tolist()
cv_results = {}

# Standard K-Fold CV
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

print("5-Fold Cross-Validation for Top 3 Models:")
print("="*60)

for model_name in top_models:
    model = models[model_name]
    
    # Create a pipeline with appropriate preprocessing
    if model_name in linear_models_list:
        # Create pipeline with linear preprocessing
        cv_pipeline = Pipeline([
            ('preprocessor', preprocessor_linear),
            ('model', model)
        ])
    elif model_name in tree_models_list:
        # Create pipeline with tree preprocessing  
        cv_pipeline = Pipeline([
            ('preprocessor', preprocessor_tree),
            ('model', model)
        ])
    else:
        # Default to linear
        cv_pipeline = Pipeline([
            ('preprocessor', preprocessor_linear),
            ('model', model)
        ])
    
    # Perform CV on original data (pipeline will handle preprocessing)
    cv_scores = cross_val_score(cv_pipeline, X, y, cv=kfold, 
                                scoring='neg_mean_absolute_error', n_jobs=-1)
    cv_scores = -cv_scores  # Convert to positive MAE
    
    cv_results[model_name] = {
        'mean': cv_scores.mean(),
        'std': cv_scores.std(),
        'min': cv_scores.min(),
        'max': cv_scores.max(),
        'scores': cv_scores
    }
    
    print(f"\n{model_name}:")
    print(f"  Mean MAE: {cv_scores.mean():.3f} (+/- {cv_scores.std():.3f})")
    print(f"  Range: [{cv_scores.min():.3f}, {cv_scores.max():.3f}]")
    print(f"  All scores: {', '.join([f'{s:.3f}' for s in cv_scores])}")

5-Fold Cross-Validation for Top 3 Models:

NeuralNet:
  Mean MAE: 0.482 (+/- 0.062)
  Range: [0.405, 0.581]
  All scores: 0.522, 0.442, 0.461, 0.581, 0.405

XGBoost:
  Mean MAE: 0.536 (+/- 0.037)
  Range: [0.506, 0.609]
  All scores: 0.510, 0.609, 0.529, 0.506, 0.524

ExtraTrees:
  Mean MAE: 0.590 (+/- 0.067)
  Range: [0.521, 0.713]
  All scores: 0.563, 0.713, 0.552, 0.602, 0.521


## 5. Prediction Intervals and Confidence Bounds

For betting, we need to know the uncertainty in our predictions.

In [10]:
# Quantile Regression for prediction intervals
from sklearn.ensemble import GradientBoostingRegressor

# Train quantile models for 10%, 50%, and 90% quantiles
quantiles = [0.1, 0.25, 0.5, 0.75, 0.9]
quantile_models = {}
quantile_predictions = {}

print("Training Quantile Regression Models:")
print("="*40)

for q in quantiles:
    print(f"Training {q:.0%} quantile model...")
    
    # Use Gradient Boosting with quantile loss
    qr_model = GradientBoostingRegressor(
        loss='quantile', alpha=q,
        n_estimators=100, max_depth=5,
        learning_rate=0.1, random_state=42
    )
    
    # Train on tree-preprocessed features (GradientBoosting is tree-based)
    qr_model.fit(X_train_tree, y_train)
    quantile_models[q] = qr_model
    
    # Make predictions
    predictions = qr_model.predict(X_test_tree)
    quantile_predictions[f'q{int(q*100)}'] = predictions

# Create prediction intervals DataFrame
prediction_intervals = pd.DataFrame(quantile_predictions, index=X_test.index)
prediction_intervals['actual'] = y_test
prediction_intervals['prediction'] = results[best_model_name]['predictions']

# Calculate interval widths
prediction_intervals['interval_80'] = prediction_intervals['q90'] - prediction_intervals['q10']
prediction_intervals['interval_50'] = prediction_intervals['q75'] - prediction_intervals['q25']

print("\nPrediction Intervals Summary:")
print(f"Mean 80% interval width: {prediction_intervals['interval_80'].mean():.2f}")
print(f"Mean 50% interval width: {prediction_intervals['interval_50'].mean():.2f}")

# Check coverage
coverage_80 = ((prediction_intervals['actual'] >= prediction_intervals['q10']) & 
               (prediction_intervals['actual'] <= prediction_intervals['q90'])).mean()
coverage_50 = ((prediction_intervals['actual'] >= prediction_intervals['q25']) & 
               (prediction_intervals['actual'] <= prediction_intervals['q75'])).mean()

print(f"\n80% interval coverage: {coverage_80:.1%} (target: 80%)")
print(f"50% interval coverage: {coverage_50:.1%} (target: 50%)")

# Display sample predictions with intervals
print("\nSample Predictions with Intervals:")
sample = prediction_intervals.head(5)[['actual', 'prediction', 'q10', 'q50', 'q90']]
print(sample.round(2))

Training Quantile Regression Models:
Training 10% quantile model...
Training 25% quantile model...
Training 50% quantile model...
Training 75% quantile model...
Training 90% quantile model...

Prediction Intervals Summary:
Mean 80% interval width: 2.50
Mean 50% interval width: 0.89

80% interval coverage: 56.4% (target: 80%)
50% interval coverage: 39.6% (target: 50%)

Sample Predictions with Intervals:
     actual  prediction    q10    q50    q90
157    4.14        4.58   3.46   3.98   4.29
239    7.17        7.23   6.00   7.79   6.89
142   26.44       26.89  18.39  28.67  29.17
177   13.68       13.78  12.80  14.87  15.63
113    6.26        6.05   6.24   5.96   6.99


## 6. Betting Line Analysis

Evaluate model performance at common betting lines.

In [ ]:
# Common PRA betting lines
betting_lines = [15.5, 20.5, 25.5, 30.5, 35.5, 40.5]

# Get best model predictions - FIX THE ERROR HERE
best_model = trained_models[best_model_name]
if best_model_name in linear_models_list:
    predictions = best_model.predict(X_test_linear)  # Fixed: use X_test_linear
elif best_model_name in tree_models_list:
    predictions = best_model.predict(X_test_tree)    # Fixed: use X_test_tree
else:
    predictions = best_model.predict(X_test_linear)  # Default to linear

# Analyze accuracy at each line
line_analysis = []

for line in betting_lines:
    # Actual outcomes
    actual_over = y_test > line
    
    # Predicted outcomes (with confidence)
    pred_over = predictions > line
    
    # Calculate metrics
    n_bets = actual_over.sum() + (~actual_over).sum()
    n_actual_overs = actual_over.sum()
    
    # Accuracy
    correct = (pred_over == actual_over).sum()
    accuracy = correct / len(y_test)
    
    # Precision for overs
    if pred_over.sum() > 0:
        precision_over = (pred_over & actual_over).sum() / pred_over.sum()
    else:
        precision_over = 0
    
    # Recall for overs
    if actual_over.sum() > 0:
        recall_over = (pred_over & actual_over).sum() / actual_over.sum()
    else:
        recall_over = 0
    
    # Calculate predicted probability using distance from line
    distances = predictions - line
    # Simple probability estimate based on distance (can be improved with calibration)
    prob_over = norm.cdf(distances, loc=0, scale=prediction_intervals['interval_50'].mean()/2)
    
    # Expected value calculation (assuming -110 odds)
    odds = 1.909  # Decimal odds for -110
    ev_over = prob_over * (odds - 1) - (1 - prob_over)
    
    # Find bets with positive EV
    positive_ev_bets = (ev_over > 0.05).sum()  # 5% edge threshold
    
    line_analysis.append({
        'Line': line,
        'Actual_Overs': n_actual_overs,
        'Actual_Over_Pct': n_actual_overs / len(y_test),
        'Accuracy': accuracy,
        'Precision_Over': precision_over,
        'Recall_Over': recall_over,
        'Avg_Prob_Over': prob_over.mean(),
        'Positive_EV_Bets': positive_ev_bets,
        'Max_EV': ev_over.max()
    })

line_df = pd.DataFrame(line_analysis)
print("Betting Line Analysis:")
print("="*80)
print(line_df.round(3))

# Identify best lines for betting
print("\nBest Lines for Betting (highest accuracy):")
best_lines = line_df.nlargest(3, 'Accuracy')[['Line', 'Accuracy', 'Positive_EV_Bets']]
print(best_lines)

## 7. Feature Importance Analysis

In [ ]:
# Get feature importance from tree-based models
tree_models = ['RandomForest', 'XGBoost', 'LightGBM', 'ExtraTrees']
importance_dict = {}

for model_name in tree_models:
    if model_name in trained_models:
        model = trained_models[model_name]
        if hasattr(model, 'feature_importances_'):
            importance_dict[model_name] = model.feature_importances_

# Create importance DataFrame
importance_df = pd.DataFrame(importance_dict, index=feature_cols)
importance_df['mean_importance'] = importance_df.mean(axis=1)
importance_df = importance_df.sort_values('mean_importance', ascending=False)

# Display top features
print("Top 10 Most Important Features:")
print("="*50)
top_features = importance_df.head(10)
for i, (feature, row) in enumerate(top_features.iterrows(), 1):
    print(f"{i:2d}. {feature:25s} - Importance: {row['mean_importance']:.4f}")

# Visualize feature importance
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Bar plot of top features
ax1 = axes[0]
top_features['mean_importance'].plot(kind='barh', ax=ax1, color='steelblue')
ax1.set_xlabel('Importance')
ax1.set_title('Top 10 Feature Importances (Average across Tree Models)')
ax1.grid(True, alpha=0.3)

# Heatmap of importance across models
ax2 = axes[1]
sns.heatmap(importance_df.head(10)[tree_models], annot=True, fmt='.3f', 
            cmap='YlOrRd', ax=ax2, cbar_kws={'label': 'Importance'})
ax2.set_title('Feature Importance by Model')
ax2.set_xlabel('Model')

plt.tight_layout()
plt.show()

# Identify stable vs unstable features
importance_std = importance_df[tree_models].std(axis=1)
print("\nMost Stable Features (consistent importance across models):")
stable_features = importance_std.nsmallest(5)
for feature, std in stable_features.items():
    mean_imp = importance_df.loc[feature, 'mean_importance']
    print(f"  {feature}: Mean={mean_imp:.4f}, Std={std:.4f}")

## 8. Error Analysis by Player Type

In [ ]:
# Combine predictions with player information - FIX PREDICTIONS DEFINITION
# Get best model predictions (fixed to use correct preprocessed data)
best_model = trained_models[best_model_name]
if best_model_name in linear_models_list:
    predictions = best_model.predict(X_test_linear)
elif best_model_name in tree_models_list:
    predictions = best_model.predict(X_test_tree)
else:
    predictions = best_model.predict(X_test_linear)

error_analysis_df = test_players.copy()
error_analysis_df['predicted_PRA'] = predictions
error_analysis_df['error'] = error_analysis_df['actual_PRA'] - error_analysis_df['predicted_PRA']
error_analysis_df['abs_error'] = np.abs(error_analysis_df['error'])
error_analysis_df['pct_error'] = (error_analysis_df['abs_error'] / error_analysis_df['actual_PRA']) * 100

# Analyze by performance tier
print("Error Analysis by Performance Tier:")
print("="*60)
tier_analysis = error_analysis_df.groupby('Performance_Tier').agg({
    'abs_error': ['mean', 'std', 'median'],
    'pct_error': ['mean', 'std'],
    'error': ['mean', 'std'],
    'Player': 'count'
}).round(2)
print(tier_analysis)

# Find hardest to predict players
print("\n10 Hardest to Predict Players:")
print("="*60)
hardest = error_analysis_df.nlargest(10, 'abs_error')[[
    'Player', 'Team', 'actual_PRA', 'predicted_PRA', 'abs_error', 'pct_error'
]]
print(hardest.round(2))

# Find most accurately predicted players
print("\n10 Most Accurately Predicted Players:")
print("="*60)
most_accurate = error_analysis_df.nsmallest(10, 'abs_error')[[
    'Player', 'Team', 'actual_PRA', 'predicted_PRA', 'abs_error', 'pct_error'
]]
print(most_accurate.round(2))

# Analyze by minutes played
error_analysis_df['minutes_category'] = pd.cut(
    error_analysis_df['MIN'], 
    bins=[0, 500, 1000, 1500, 2000, 3000],
    labels=['<500', '500-1000', '1000-1500', '1500-2000', '2000+']
)

print("\nError Analysis by Minutes Played:")
print("="*60)
minutes_analysis = error_analysis_df.groupby('minutes_category').agg({
    'abs_error': ['mean', 'std'],
    'pct_error': 'mean',
    'Player': 'count'
}).round(2)
print(minutes_analysis)

## 9. Visualization of Results

In [ ]:
# Create comprehensive visualization - FIX PREDICTIONS FIRST
# Get best model predictions (fixed to use correct preprocessed data)
best_model = trained_models[best_model_name]
if best_model_name in linear_models_list:
    predictions = best_model.predict(X_test_linear)
elif best_model_name in tree_models_list:
    predictions = best_model.predict(X_test_tree)
else:
    predictions = best_model.predict(X_test_linear)

fig = plt.figure(figsize=(20, 12))
gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)

# 1. Actual vs Predicted scatter
ax1 = fig.add_subplot(gs[0, 0])
ax1.scatter(y_test, predictions, alpha=0.5, s=30)
ax1.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
ax1.set_xlabel('Actual PRA')
ax1.set_ylabel('Predicted PRA')
ax1.set_title(f'Predictions vs Actual ({best_model_name})')
ax1.grid(True, alpha=0.3)
ax1.text(0.05, 0.95, f'R² = {results[best_model_name]["test_r2"]:.3f}',
         transform=ax1.transAxes, verticalalignment='top')

# 2. Residual plot
ax2 = fig.add_subplot(gs[0, 1])
residuals = y_test - predictions
ax2.scatter(predictions, residuals, alpha=0.5, s=30)
ax2.axhline(y=0, color='r', linestyle='--')
ax2.set_xlabel('Predicted PRA')
ax2.set_ylabel('Residuals')
ax2.set_title('Residual Plot')
ax2.grid(True, alpha=0.3)

# 3. Error distribution
ax3 = fig.add_subplot(gs[0, 2])
ax3.hist(residuals, bins=30, edgecolor='black', alpha=0.7)
ax3.axvline(x=0, color='r', linestyle='--')
ax3.set_xlabel('Prediction Error')
ax3.set_ylabel('Frequency')
ax3.set_title('Error Distribution')
ax3.grid(True, alpha=0.3)
ax3.text(0.05, 0.95, f'Mean: {residuals.mean():.2f}\nStd: {residuals.std():.2f}',
         transform=ax3.transAxes, verticalalignment='top')

# 4. Model comparison
ax4 = fig.add_subplot(gs[1, :])
model_names = list(results.keys())
test_maes = [results[m]['test_mae'] for m in model_names]
colors = ['green' if m == best_model_name else 'steelblue' for m in model_names]
bars = ax4.bar(model_names, test_maes, color=colors)
ax4.set_xlabel('Model')
ax4.set_ylabel('Test MAE')
ax4.set_title('Model Performance Comparison')
ax4.grid(True, alpha=0.3, axis='y')
ax4.set_xticklabels(model_names, rotation=45, ha='right')

# Add value labels on bars
for bar, mae in zip(bars, test_maes):
    height = bar.get_height()
    ax4.text(bar.get_x() + bar.get_width()/2., height,
             f'{mae:.2f}', ha='center', va='bottom')

# 5. Prediction intervals
ax5 = fig.add_subplot(gs[2, 0])
sample_idx = np.random.choice(len(prediction_intervals), 20, replace=False)
sample_data = prediction_intervals.iloc[sample_idx].sort_values('actual')
x_pos = range(len(sample_data))

ax5.scatter(x_pos, sample_data['actual'], color='red', s=50, label='Actual', zorder=5)
ax5.scatter(x_pos, sample_data['prediction'], color='blue', s=30, label='Predicted', zorder=4)
ax5.fill_between(x_pos, sample_data['q10'], sample_data['q90'], 
                  alpha=0.3, color='gray', label='80% PI')
ax5.fill_between(x_pos, sample_data['q25'], sample_data['q75'], 
                  alpha=0.5, color='gray', label='50% PI')
ax5.set_xlabel('Sample Players')
ax5.set_ylabel('PRA')
ax5.set_title('Prediction Intervals (20 Random Players)')
ax5.legend()
ax5.grid(True, alpha=0.3)

# 6. Error by tier (need to define error_analysis_df first)
ax6 = fig.add_subplot(gs[2, 1])
# Create error_analysis_df if not exists
if 'error_analysis_df' not in locals():
    error_analysis_df = test_players.copy()
    error_analysis_df['predicted_PRA'] = predictions
    error_analysis_df['abs_error'] = np.abs(error_analysis_df['actual_PRA'] - error_analysis_df['predicted_PRA'])

tier_errors = error_analysis_df.groupby('Performance_Tier')['abs_error'].mean().sort_values()
tier_errors.plot(kind='barh', ax=ax6, color='coral')
ax6.set_xlabel('Mean Absolute Error')
ax6.set_title('Prediction Error by Performance Tier')
ax6.grid(True, alpha=0.3, axis='x')

# 7. Calibration plot for betting
ax7 = fig.add_subplot(gs[2, 2])
# For a specific line (e.g., 25.5)
test_line = 25.5
distances = predictions - test_line
prob_over = norm.cdf(distances, loc=0, scale=prediction_intervals['interval_50'].mean()/2)

# Bin probabilities
n_bins = 10
bin_edges = np.linspace(0, 1, n_bins + 1)
bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
actual_freq = []
predicted_freq = []

for i in range(n_bins):
    mask = (prob_over >= bin_edges[i]) & (prob_over < bin_edges[i+1])
    if mask.sum() > 0:
        actual_freq.append((y_test[mask] > test_line).mean())
        predicted_freq.append(prob_over[mask].mean())
    else:
        actual_freq.append(np.nan)
        predicted_freq.append(np.nan)

ax7.scatter(predicted_freq, actual_freq, s=50)
ax7.plot([0, 1], [0, 1], 'r--', lw=2)
ax7.set_xlabel('Predicted Probability')
ax7.set_ylabel('Actual Frequency')
ax7.set_title(f'Calibration Plot (Line: {test_line})')
ax7.grid(True, alpha=0.3)
ax7.set_xlim([0, 1])
ax7.set_ylim([0, 1])

plt.suptitle('NBA Props Model Evaluation Dashboard', fontsize=16, y=1.02)
plt.show()

## 10. Production-Ready Predictions

Generate betting recommendations with confidence levels.

In [ ]:
def generate_betting_recommendations(player_features, model, scaler, quantile_models, 
                                     line, min_edge=0.05, odds=-110):
    """
    Generate betting recommendations for a given line.
    
    Parameters:
    -----------
    player_features : DataFrame
        Features for players to predict
    model : sklearn model
        Trained prediction model
    scaler : sklearn scaler
        Feature scaler
    quantile_models : dict
        Quantile regression models
    line : float
        Betting line
    min_edge : float
        Minimum edge required for recommendation
    odds : int
        American odds (e.g., -110)
    """
    
    # Convert American odds to decimal
    if odds < 0:
        decimal_odds = 1 + (100 / abs(odds))
    else:
        decimal_odds = 1 + (odds / 100)
    
    # Make predictions
    if model.__class__.__name__ in ['Ridge', 'Lasso', 'ElasticNet', 'Huber', 'SVR', 'MLPRegressor']:
        X_scaled = scaler.transform(player_features)
        predictions = model.predict(X_scaled)
    else:
        predictions = model.predict(player_features)
    
    # Get prediction intervals
    q10 = quantile_models[0.1].predict(player_features)
    q90 = quantile_models[0.9].predict(player_features)
    
    # Calculate probability of going over
    std_estimate = (q90 - q10) / 2.56  # Approximate std from 80% interval
    prob_over = norm.cdf((predictions - line) / std_estimate)
    prob_under = 1 - prob_over
    
    # Calculate expected value
    ev_over = prob_over * (decimal_odds - 1) - prob_under
    ev_under = prob_under * (decimal_odds - 1) - prob_over
    
    # Create recommendations DataFrame
    recommendations = pd.DataFrame({
        'prediction': predictions,
        'line': line,
        'prob_over': prob_over,
        'prob_under': prob_under,
        'ev_over': ev_over,
        'ev_under': ev_under,
        'confidence_interval_80': f"[{q10:.1f}, {q90:.1f}]",
        'recommendation': 'NO BET'
    }, index=player_features.index)
    
    # Make recommendations
    recommendations.loc[ev_over > min_edge, 'recommendation'] = 'BET OVER'
    recommendations.loc[ev_under > min_edge, 'recommendation'] = 'BET UNDER'
    
    # Add confidence level
    max_ev = np.maximum(ev_over, ev_under)
    recommendations['confidence'] = 'LOW'
    recommendations.loc[max_ev > 0.10, 'confidence'] = 'MEDIUM'
    recommendations.loc[max_ev > 0.15, 'confidence'] = 'HIGH'
    recommendations.loc[max_ev > 0.20, 'confidence'] = 'VERY HIGH'
    
    return recommendations

# Generate recommendations for test set
test_line = 25.5
recommendations = generate_betting_recommendations(
    X_test, trained_models[best_model_name], scaler, 
    quantile_models, test_line, min_edge=0.05
)

# Add player info
recommendations = pd.concat([test_players[['Player', 'Team', 'actual_PRA']], recommendations], axis=1)

# Display betting opportunities
betting_opportunities = recommendations[recommendations['recommendation'] != 'NO BET'].copy()
betting_opportunities = betting_opportunities.sort_values('confidence', 
                                                          ascending=False,
                                                          key=lambda x: x.map({'VERY HIGH': 4, 
                                                                              'HIGH': 3, 
                                                                              'MEDIUM': 2, 
                                                                              'LOW': 1}))

print(f"Betting Recommendations for Line: {test_line}")
print("="*80)
print(f"Total opportunities found: {len(betting_opportunities)}")
print(f"High confidence bets: {(betting_opportunities['confidence'].isin(['HIGH', 'VERY HIGH'])).sum()}")

print("\nTop 10 Betting Opportunities:")
display_cols = ['Player', 'Team', 'actual_PRA', 'prediction', 'recommendation', 
                'prob_over', 'ev_over', 'confidence']
print(betting_opportunities.head(10)[display_cols].round(3))

# Evaluate recommendations
if len(betting_opportunities) > 0:
    # Check actual results
    correct_bets = 0
    for idx, row in betting_opportunities.iterrows():
        if row['recommendation'] == 'BET OVER' and row['actual_PRA'] > test_line:
            correct_bets += 1
        elif row['recommendation'] == 'BET UNDER' and row['actual_PRA'] < test_line:
            correct_bets += 1
    
    accuracy = correct_bets / len(betting_opportunities)
    print(f"\nBacktest Results:")
    print(f"Accuracy: {accuracy:.1%} ({correct_bets}/{len(betting_opportunities)})")
    
    # ROI calculation
    profit = correct_bets * 0.909 - (len(betting_opportunities) - correct_bets)  # Assuming unit bets
    roi = (profit / len(betting_opportunities)) * 100
    print(f"ROI: {roi:.1f}%")

## 11. Model Ensemble and Stacking

In [ ]:
# Create ensemble predictions
from sklearn.ensemble import VotingRegressor, StackingRegressor

# Select top 3 models for ensemble
top_3_models = comparison_df.head(3).index.tolist()
print(f"Creating ensemble with top 3 models: {top_3_models}")

# Prepare base models
base_models = []
for model_name in top_3_models:
    base_models.append((model_name, models[model_name]))

# Create voting ensemble (average predictions)
voting_ensemble = VotingRegressor(estimators=base_models)

# Train voting ensemble - FIX: use appropriate preprocessing
print("\nTraining Voting Ensemble...")
# Check if we need linear or tree preprocessing
needs_linear = any(name in linear_models_list for name in top_3_models)
needs_tree = any(name in tree_models_list for name in top_3_models)

if needs_linear and not needs_tree:
    # All models need linear preprocessing
    voting_ensemble.fit(X_train_linear, y_train)
    voting_pred = voting_ensemble.predict(X_test_linear)
elif needs_tree and not needs_linear:
    # All models need tree preprocessing
    voting_ensemble.fit(X_train_tree, y_train)
    voting_pred = voting_ensemble.predict(X_test_tree)
else:
    # Mixed - use linear as default since it has more features
    print("WARNING: Mixed model types in ensemble - using linear preprocessing")
    voting_ensemble.fit(X_train_linear, y_train)
    voting_pred = voting_ensemble.predict(X_test_linear)

voting_mae = mean_absolute_error(y_test, voting_pred)
voting_r2 = r2_score(y_test, voting_pred)
print(f"Voting Ensemble - MAE: {voting_mae:.3f}, R²: {voting_r2:.3f}")

# Create stacking ensemble
print("\nTraining Stacking Ensemble...")
stacking_ensemble = StackingRegressor(
    estimators=base_models,
    final_estimator=Ridge(alpha=1.0),
    cv=5  # Use cross-validation to train meta-model
)

# Train stacking ensemble - use same logic as voting
if needs_linear and not needs_tree:
    stacking_ensemble.fit(X_train_linear, y_train)
    stacking_pred = stacking_ensemble.predict(X_test_linear)
elif needs_tree and not needs_linear:
    stacking_ensemble.fit(X_train_tree, y_train)
    stacking_pred = stacking_ensemble.predict(X_test_tree)
else:
    print("WARNING: Mixed model types in ensemble - using linear preprocessing")
    stacking_ensemble.fit(X_train_linear, y_train)
    stacking_pred = stacking_ensemble.predict(X_test_linear)

stacking_mae = mean_absolute_error(y_test, stacking_pred)
stacking_r2 = r2_score(y_test, stacking_pred)
print(f"Stacking Ensemble - MAE: {stacking_mae:.3f}, R²: {stacking_r2:.3f}")

# Compare all models including ensembles
print("\nFinal Model Comparison:")
print("="*50)
final_comparison = pd.DataFrame({
    'Model': [best_model_name, 'Voting Ensemble', 'Stacking Ensemble'],
    'MAE': [results[best_model_name]['test_mae'], voting_mae, stacking_mae],
    'R²': [results[best_model_name]['test_r2'], voting_r2, stacking_r2]
}).sort_values('MAE')
print(final_comparison)

# Visualize ensemble performance
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Best single model
axes[0].scatter(y_test, results[best_model_name]['predictions'], alpha=0.5)
axes[0].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
axes[0].set_title(f'{best_model_name}\nMAE: {results[best_model_name]["test_mae"]:.3f}')
axes[0].set_xlabel('Actual')
axes[0].set_ylabel('Predicted')
axes[0].grid(True, alpha=0.3)

# Voting ensemble
axes[1].scatter(y_test, voting_pred, alpha=0.5)
axes[1].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
axes[1].set_title(f'Voting Ensemble\nMAE: {voting_mae:.3f}')
axes[1].set_xlabel('Actual')
axes[1].set_ylabel('Predicted')
axes[1].grid(True, alpha=0.3)

# Stacking ensemble
axes[2].scatter(y_test, stacking_pred, alpha=0.5)
axes[2].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
axes[2].set_title(f'Stacking Ensemble\nMAE: {stacking_mae:.3f}')
axes[2].set_xlabel('Actual')
axes[2].set_ylabel('Predicted')
axes[2].grid(True, alpha=0.3)

plt.suptitle('Ensemble Model Comparison', fontsize=14)
plt.tight_layout()
plt.show()

## 12. Save Production Model

In [ ]:
import joblib
import json
from datetime import datetime

# Determine best final model
best_final_model = None
best_final_name = None
best_final_mae = float('inf')

if voting_mae < best_final_mae:
    best_final_model = voting_ensemble
    best_final_name = 'VotingEnsemble'
    best_final_mae = voting_mae

if stacking_mae < best_final_mae:
    best_final_model = stacking_ensemble
    best_final_name = 'StackingEnsemble'
    best_final_mae = stacking_mae

if results[best_model_name]['test_mae'] < best_final_mae:
    best_final_model = trained_models[best_model_name]
    best_final_name = best_model_name
    best_final_mae = results[best_model_name]['test_mae']

# Create models directory
models_dir = Path('/Users/diyagamah/Documents/nba_props_model/models')
models_dir.mkdir(exist_ok=True)

# Save model artifacts
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
model_filename = models_dir / f'pra_model_{best_final_name}_{timestamp}.pkl'
scaler_filename = models_dir / f'scaler_{timestamp}.pkl'
quantiles_filename = models_dir / f'quantile_models_{timestamp}.pkl'
metadata_filename = models_dir / f'model_metadata_{timestamp}.json'

# Save files
joblib.dump(best_final_model, model_filename)
joblib.dump(scaler, scaler_filename)
joblib.dump(quantile_models, quantiles_filename)

# Save metadata
metadata = {
    'model_type': best_final_name,
    'test_mae': float(best_final_mae),
    'test_r2': float(voting_r2 if best_final_name == 'VotingEnsemble' else 
                    stacking_r2 if best_final_name == 'StackingEnsemble' else 
                    results[best_model_name]['test_r2']),
    'features': feature_cols,
    'n_training_samples': len(X_train),
    'n_test_samples': len(X_test),
    'timestamp': timestamp,
    'target_mean': float(y_train.mean()),
    'target_std': float(y_train.std())
}

with open(metadata_filename, 'w') as f:
    json.dump(metadata, f, indent=2)

print(f"Production model saved:")
print(f"  Model: {model_filename}")
print(f"  Scaler: {scaler_filename}")
print(f"  Quantiles: {quantiles_filename}")
print(f"  Metadata: {metadata_filename}")
print(f"\nBest Model: {best_final_name}")
print(f"Test MAE: {best_final_mae:.3f}")

## Summary and Next Steps

### Model Performance:
- Best single model and performance metrics
- Ensemble improvements
- Confidence intervals and calibration

### Betting Insights:
- Most profitable betting lines
- Player types with highest prediction accuracy
- Expected value calculations

### Production Deployment:
1. Model and artifacts saved for production use
2. Feature importance identified for monitoring
3. Error patterns understood for risk management

### Recommended Next Steps:
1. **Backtesting**: Test on historical betting lines
2. **Live monitoring**: Track model performance on new games
3. **Feature engineering**: Add game-specific features (opponent, home/away, rest days)
4. **Calibration**: Improve probability estimates for betting lines
5. **Risk management**: Implement Kelly criterion for bet sizing